# 기존

In [1]:
import numpy as np
import pandas as pd
from PIL import Image
import tensorflow as tf
from keras import Input
import sys,os,math,pathlib
from tensorflow import keras
from keras import models, layers
import matplotlib.pyplot as plt
from tensorflow.python.client import device_lib
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from keras.models import Model, load_model
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers, initializers, regularizers, metrics
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import BatchNormalization, Conv2D, Activation, Dense, GlobalAveragePooling2D, MaxPooling2D, ZeroPadding2D, Add

In [23]:
batch_size = 32
split_percent=0.3
rand_seed=123
shuffle_num=1000
EPOCHS=100  # add_class test
dir_path = "E:/##kpu_capstone_voice_data/!snowflo_real_final/saddummy@gmail.com_augment_add_class"

### 현재 데이터 클래스 1개 추가된 상태

In [24]:
data_dir = pathlib.Path(dir_path)
image_list = list(data_dir.glob('*/*.png'))
image_size = Image.open(image_list[0]).size
img_width=image_size[0]
img_height=image_size[1]

In [25]:
print(image_list[0])

E:\##kpu_capstone_voice_data\!snowflo_real_final\saddummy@gmail.com_augment_add_class\3상호한테전화해줘\sangho (136).png


In [26]:
print("{} {}".format(img_width, img_height))

260 200


### 데이터 셋 만들기 코드 (기존)

In [27]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=split_percent,
    subset="training",
    seed=rand_seed,
    image_size=(img_height, img_width),
    batch_size=batch_size)

Found 1790 files belonging to 8 classes.
Using 1253 files for training.


In [28]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=split_percent ,
    subset="validation",
    seed=rand_seed,
    image_size=(img_height, img_width),
    batch_size=batch_size)

Found 1790 files belonging to 8 classes.
Using 537 files for validation.


In [29]:
class_names = train_ds.class_names
num_classes = len(class_names)
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_ds = train_ds.cache().shuffle(shuffle_num).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
normalization_layer = layers.experimental.preprocessing.Rescaling(1./255)
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))

In [30]:
class_names

['1굿모닝',
 '2내일날씨어때',
 '3상호한테전화해줘',
 '4오늘날씨어때',
 '5유튜브에서동빈나틀어줘',
 '6음악추천해줘',
 '7출근길교통상황어때',
 '8티비켜줘']

In [9]:
K = num_classes

input_tensor = Input(shape=(img_height, img_width, 3), dtype='float32', name='input')
def conv1_layer(x):    
    x = ZeroPadding2D(padding=(3, 3))(x)
    x = Conv2D(64, (7, 7), strides=(2, 2))(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = ZeroPadding2D(padding=(1,1))(x) 
    return x    
def conv2_layer(x):         
    x = MaxPooling2D((3, 3), 2)(x)      
    shortcut = x
    for i in range(3):
        if (i == 0):
            x = Conv2D(64, (1, 1), strides=(1, 1), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
            
            x = Conv2D(64, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
 
            x = Conv2D(256, (1, 1), strides=(1, 1), padding='valid')(x)
            shortcut = Conv2D(256, (1, 1), strides=(1, 1), padding='valid')(shortcut)            
            x = BatchNormalization()(x)
            shortcut = BatchNormalization()(shortcut)
 
            x = Add()([x, shortcut])
            x = Activation('relu')(x)
            
            shortcut = x
 
        else:
            x = Conv2D(64, (1, 1), strides=(1, 1), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
            
            x = Conv2D(64, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
 
            x = Conv2D(256, (1, 1), strides=(1, 1), padding='valid')(x)
            x = BatchNormalization()(x)            
 
            x = Add()([x, shortcut])   
            x = Activation('relu')(x)  
            shortcut = x            
    return x
 
def conv3_layer(x):        
    shortcut = x    
    
    for i in range(4):     
        if(i == 0):            
            x = Conv2D(128, (1, 1), strides=(2, 2), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)        
            
            x = Conv2D(128, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)  
 
            x = Conv2D(512, (1, 1), strides=(1, 1), padding='valid')(x)
            shortcut = Conv2D(512, (1, 1), strides=(2, 2), padding='valid')(shortcut)
            x = BatchNormalization()(x)
            shortcut = BatchNormalization()(shortcut)            
 
            x = Add()([x, shortcut])    
            x = Activation('relu')(x)    
 
            shortcut = x              
        
        else:
            x = Conv2D(128, (1, 1), strides=(1, 1), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
            
            x = Conv2D(128, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
 
            x = Conv2D(512, (1, 1), strides=(1, 1), padding='valid')(x)
            x = BatchNormalization()(x)            
 
            x = Add()([x, shortcut])     
            x = Activation('relu')(x)
 
            shortcut = x      
            
    return x

def conv4_layer(x):
    shortcut = x        
  
    for i in range(6):     
        if(i == 0):            
            x = Conv2D(256, (1, 1), strides=(2, 2), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)        
            
            x = Conv2D(256, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)  
 
            x = Conv2D(1024, (1, 1), strides=(1, 1), padding='valid')(x)
            shortcut = Conv2D(1024, (1, 1), strides=(2, 2), padding='valid')(shortcut)
            x = BatchNormalization()(x)
            shortcut = BatchNormalization()(shortcut)
 
            x = Add()([x, shortcut]) 
            x = Activation('relu')(x)
 
            shortcut = x               
        
        else:
            x = Conv2D(256, (1, 1), strides=(1, 1), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
            
            x = Conv2D(256, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
 
            x = Conv2D(1024, (1, 1), strides=(1, 1), padding='valid')(x)
            x = BatchNormalization()(x)            
 
            x = Add()([x, shortcut])    
            x = Activation('relu')(x)
 
            shortcut = x      
 
    return x

def conv5_layer(x):
    shortcut = x    
  
    for i in range(3):     
        if(i == 0):            
            x = Conv2D(512, (1, 1), strides=(2, 2), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)        
            
            x = Conv2D(512, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)  
 
            x = Conv2D(2048, (1, 1), strides=(1, 1), padding='valid')(x)
            shortcut = Conv2D(2048, (1, 1), strides=(2, 2), padding='valid')(shortcut)
            x = BatchNormalization()(x)
            shortcut = BatchNormalization()(shortcut)            
 
            x = Add()([x, shortcut])  
            x = Activation('relu')(x)      
 
            shortcut = x               
        
        else:
            x = Conv2D(512, (1, 1), strides=(1, 1), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
            
            x = Conv2D(512, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
 
            x = Conv2D(2048, (1, 1), strides=(1, 1), padding='valid')(x)
            x = BatchNormalization()(x)           
            
            x = Add()([x, shortcut]) 
            x = Activation('relu')(x)       
 
            shortcut = x                  
 
    return x

x = conv1_layer(input_tensor)
x = conv2_layer(x)
x = conv3_layer(x)
x = conv4_layer(x)
x = conv5_layer(x)
x = GlobalAveragePooling2D()(x)
output_tensor = Dense(K, activation='softmax')(x)  # dense
resnet50 = Model(input_tensor, output_tensor)
resnet50.summary()

resnet50.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 200, 260, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 206, 266, 3)  0           input[0][0]                      
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 100, 130, 64) 9472        zero_padding2d[0][0]             
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 100, 130, 64) 256         conv2d[0][0]                     
______________________________________________________________________________________________

__________________________________________________________________________________________________
batch_normalization_45 (BatchNo (None, 7, 9, 2048)   8192        conv2d_45[0][0]                  
__________________________________________________________________________________________________
batch_normalization_46 (BatchNo (None, 7, 9, 2048)   8192        conv2d_46[0][0]                  
__________________________________________________________________________________________________
add_13 (Add)                    (None, 7, 9, 2048)   0           batch_normalization_45[0][0]     
                                                                 batch_normalization_46[0][0]     
__________________________________________________________________________________________________
activation_42 (Activation)      (None, 7, 9, 2048)   0           add_13[0][0]                     
__________________________________________________________________________________________________
conv2d_47 

In [ ]:
EPOCHS = 150  # 1차 학습
with tf.device('/GPU:0'):
    history = resnet50.fit(
        train_ds,
        validation_data=val_ds,
        epochs=EPOCHS,
    )

### 변경사항

In [12]:
# # save_format='tf' 로 저장
# from keras.models import load_model
# resnet50.save('E:/##kpu_capstone_voice_data/##model/saddummy@gmail.com_augment', save_format='tf')

INFO:tensorflow:Assets written to: E:/##kpu_capstone_voice_data/##model/saddummy@gmail.com_augment\assets


In [13]:
# save_format='h5' 로 저장
from keras.models import load_model
resnet50.save('E:/##kpu_capstone_voice_data/##model/saddummy@gmail.com_augment.h5')

### 학습모드 클래스 추가 없는 경우

In [12]:
# h5 model 로 loadfrom keras.models import load_model
test_h5 = load_model('E:/##kpu_capstone_voice_data/##model/saddummy@gmail.com_augment_mk_TV1.h5')

In [13]:
weights_bak = test_h5.layers[-1].get_weights()  # dense layer weight 저장
weights_bak

[array([[-0.01450044, -0.03656423,  0.02314706, ...,  0.02294294,
         -0.04593412, -0.01897183],
        [-0.04334139, -0.03761547, -0.0315095 , ..., -0.01010539,
          0.05081044,  0.06624503],
        [ 0.02740716,  0.0543118 , -0.06785501, ..., -0.0299793 ,
          0.02350104, -0.0502148 ],
        ...,
        [-0.02241648, -0.05700171,  0.0259869 , ...,  0.05745413,
         -0.08571246, -0.01126638],
        [ 0.00181318, -0.06370977, -0.0103215 , ..., -0.02639685,
          0.05047071, -0.01829127],
        [-0.02944381,  0.00647926, -0.05704192, ..., -0.04779259,
          0.04159585,  0.04488668]], dtype=float32),
 array([ 9.3754046e-03, -1.4954056e-03, -1.3052986e-03,  1.6418620e-03,
         1.6421090e-03, -4.3301987e-03, -8.6039985e-03, -5.1402181e-05],
       dtype=float32)]

In [14]:
# 기존 모델의 다른 레이어층 가중치 봉인
for layer in test_h5.layers[:-1]:
    layer.trainable = False

In [15]:
# 마지막 레이어 dense layer 확인  (추가 클래스 없음)
dense_input = test_h5.layers[-2].output

output_layer = test_h5.layers[-1]  # dense
output_layer.output.shape


TensorShape([None, 8])

In [16]:
# 이어학습 (전이학습)
# https://codetorial.net/tensorflow/transfer_learning.html
EPOCHS = 10
history = test_h5.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
)

Epoch 1/10
40/40 [==============================] - 36s 552ms/step - loss: 0.1091 - accuracy: 0.9737 - val_loss: 0.1063 - val_accuracy: 0.9758
Epoch 2/10
40/40 [==============================] - 9s 226ms/step - loss: 0.1055 - accuracy: 0.9800 - val_loss: 0.0823 - val_accuracy: 0.9814
Epoch 3/10
40/40 [==============================] - 9s 226ms/step - loss: 0.0909 - accuracy: 0.9769 - val_loss: 0.0732 - val_accuracy: 0.9777
Epoch 4/10
40/40 [==============================] - 9s 228ms/step - loss: 0.0693 - accuracy: 0.9824 - val_loss: 0.0623 - val_accuracy: 0.9814
Epoch 5/10
40/40 [==============================] - 9s 229ms/step - loss: 0.0723 - accuracy: 0.9816 - val_loss: 0.0638 - val_accuracy: 0.9832
Epoch 6/10
40/40 [==============================] - 9s 225ms/step - loss: 0.0511 - accuracy: 0.9816 - val_loss: 0.0792 - val_accuracy: 0.9777
Epoch 7/10
40/40 [==============================] - 9s 225ms/step - loss: 0.0453 - accuracy: 0.9856 - val_loss: 0.0599 - val_accuracy: 0.9777
Epoch

In [17]:
# save_format='h5' 로 저장
from keras.models import load_model
test_h5.save('E:/##kpu_capstone_voice_data/##model/saddummy@gmail.com_augment_mk_TV2.h5')

### 학습모드 클래스 추가 있는 경우

In [31]:
# h5 model 로 loadfrom keras.models import load_model
test_h5 = load_model('E:/##kpu_capstone_voice_data/##model/saddummy@gmail.com_augment_mk2.h5')

In [34]:
test_h5.layers[-1].output.shape

TensorShape([None, 7])

In [32]:
# 마지막 레이어 dense layer 확인 (추가 클래스 있음)
K = num_classes # = len(class_names) # 데이터 셋 정의시에 사용 
dense_input = test_h5.layers[-2].output

# weights_back = test_h5.layers[-1].get_weights()
# weights_back
add_output_layer = Dense(K, activation='softmax')(dense_input)  # dense
add_output_layer.shape

TensorShape([None, 8])

In [35]:
# 가중치를 이어받은 새로운 모델 생성 # 기존 모델의 다른 레이어층 가중치 봉인
# 모델 생성 dense layer 확인  (추가 클래스 있음)
resume_model = Model(test_h5.input, outputs=add_output_layer)
resume_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 200, 260, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 206, 266, 3)  0           input[0][0]                      
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 100, 130, 64) 9472        zero_padding2d[0][0]             
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 100, 130, 64) 256         conv2d[0][0]                     
______________________________________________________________________________________________

__________________________________________________________________________________________________
batch_normalization_45 (BatchNo (None, 7, 9, 2048)   8192        conv2d_45[0][0]                  
__________________________________________________________________________________________________
batch_normalization_46 (BatchNo (None, 7, 9, 2048)   8192        conv2d_46[0][0]                  
__________________________________________________________________________________________________
add_13 (Add)                    (None, 7, 9, 2048)   0           batch_normalization_45[0][0]     
                                                                 batch_normalization_46[0][0]     
__________________________________________________________________________________________________
activation_42 (Activation)      (None, 7, 9, 2048)   0           add_13[0][0]                     
__________________________________________________________________________________________________
conv2d_47 

In [36]:
# 이전 모델 클래스 수 - 현재 모델 클래스 수
diff = test_h5.layers[-1].get_weights()[1].shape[0] - num_classes
diff

-1

In [37]:
weights_bak = test_h5.layers[-1].get_weights()
weights_bak

[array([[-0.01758778, -0.0401866 ,  0.02128228, ..., -0.00992273,
          0.02440113, -0.04194269],
        [-0.04484023, -0.02908303, -0.03195988, ...,  0.03778315,
         -0.00179854,  0.0549266 ],
        [ 0.02572325,  0.04758549, -0.04696421, ...,  0.01152843,
         -0.0313897 ,  0.02605276],
        ...,
        [-0.01317754, -0.04254365,  0.02687128, ...,  0.0224937 ,
          0.05028516, -0.05951944],
        [ 0.02053918, -0.02275643, -0.00458246, ...,  0.01485057,
          0.0006123 ,  0.04798047],
        [-0.04339123,  0.01498608, -0.04606346, ...,  0.03704583,
         -0.03531209,  0.05291448]], dtype=float32),
 array([ 0.00156262, -0.0005003 , -0.0001414 ,  0.00160149,  0.00075813,
         0.00143371, -0.00171568], dtype=float32)]

In [38]:
weights_new = resume_model.layers[-1].get_weights()
weights_new

[array([[-0.01529003, -0.02451769, -0.05343588, ..., -0.0263364 ,
         -0.02566781, -0.0176605 ],
        [-0.05038335,  0.02309452, -0.04776579, ..., -0.01856512,
          0.04304143,  0.04465139],
        [-0.00961107,  0.01339429,  0.01627303, ...,  0.05370007,
         -0.01249605,  0.02416213],
        ...,
        [-0.01354912, -0.0069397 , -0.03320608, ...,  0.00185799,
         -0.03208883, -0.04914891],
        [-0.03977009, -0.04182366, -0.05346533, ..., -0.0271089 ,
          0.02171285, -0.05371628],
        [-0.0522292 ,  0.03493313, -0.03463691, ..., -0.01809295,
          0.01217355, -0.04758233]], dtype=float32),
 array([0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)]

In [39]:
# 가중치 백업
weights_new[0][:, :diff] = weights_bak[0]
weights_new[1][:diff] = weights_bak[1]
weights_new

[array([[-0.01758778, -0.0401866 ,  0.02128228, ...,  0.02440113,
         -0.04194269, -0.0176605 ],
        [-0.04484023, -0.02908303, -0.03195988, ..., -0.00179854,
          0.0549266 ,  0.04465139],
        [ 0.02572325,  0.04758549, -0.04696421, ..., -0.0313897 ,
          0.02605276,  0.02416213],
        ...,
        [-0.01317754, -0.04254365,  0.02687128, ...,  0.05028516,
         -0.05951944, -0.04914891],
        [ 0.02053918, -0.02275643, -0.00458246, ...,  0.0006123 ,
          0.04798047, -0.05371628],
        [-0.04339123,  0.01498608, -0.04606346, ..., -0.03531209,
          0.05291448, -0.04758233]], dtype=float32),
 array([ 0.00156262, -0.0005003 , -0.0001414 ,  0.00160149,  0.00075813,
         0.00143371, -0.00171568,  0.        ], dtype=float32)]

In [40]:
print(resume_model.layers[-1].get_weights())
resume_model.layers[-1].set_weights(weights_new)
print(resume_model.layers[-1].get_weights())

[array([[-0.01529003, -0.02451769, -0.05343588, ..., -0.0263364 ,
        -0.02566781, -0.0176605 ],
       [-0.05038335,  0.02309452, -0.04776579, ..., -0.01856512,
         0.04304143,  0.04465139],
       [-0.00961107,  0.01339429,  0.01627303, ...,  0.05370007,
        -0.01249605,  0.02416213],
       ...,
       [-0.01354912, -0.0069397 , -0.03320608, ...,  0.00185799,
        -0.03208883, -0.04914891],
       [-0.03977009, -0.04182366, -0.05346533, ..., -0.0271089 ,
         0.02171285, -0.05371628],
       [-0.0522292 ,  0.03493313, -0.03463691, ..., -0.01809295,
         0.01217355, -0.04758233]], dtype=float32), array([0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)]
[array([[-0.01758778, -0.0401866 ,  0.02128228, ...,  0.02440113,
        -0.04194269, -0.0176605 ],
       [-0.04484023, -0.02908303, -0.03195988, ..., -0.00179854,
         0.0549266 ,  0.04465139],
       [ 0.02572325,  0.04758549, -0.04696421, ..., -0.0313897 ,
         0.02605276,  0.02416213],
       ...,
 

In [41]:
# 가중치 봉인
for layer in resume_model.layers[:-1]:
    layer.trainable = False

In [42]:
# 컴파일
resume_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

resume_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 200, 260, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 206, 266, 3)  0           input[0][0]                      
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 100, 130, 64) 9472        zero_padding2d[0][0]             
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 100, 130, 64) 256         conv2d[0][0]                     
______________________________________________________________________________________________

__________________________________________________________________________________________________
activation_42 (Activation)      (None, 7, 9, 2048)   0           add_13[0][0]                     
__________________________________________________________________________________________________
conv2d_47 (Conv2D)              (None, 7, 9, 512)    1049088     activation_42[0][0]              
__________________________________________________________________________________________________
batch_normalization_47 (BatchNo (None, 7, 9, 512)    2048        conv2d_47[0][0]                  
__________________________________________________________________________________________________
activation_43 (Activation)      (None, 7, 9, 512)    0           batch_normalization_47[0][0]     
__________________________________________________________________________________________________
conv2d_48 (Conv2D)              (None, 7, 9, 512)    2359808     activation_43[0][0]              
__________

In [43]:
# 이어학습 (전이학습)
# https://codetorial.net/tensorflow/transfer_learning.html
EPOCHS = 10
history = resume_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
)

Epoch 1/10
40/40 [==============================] - 24s 467ms/step - loss: 0.4161 - accuracy: 0.9554 - val_loss: 0.1779 - val_accuracy: 0.9683
Epoch 2/10
40/40 [==============================] - 9s 229ms/step - loss: 0.1495 - accuracy: 0.9725 - val_loss: 0.1143 - val_accuracy: 0.9721
Epoch 3/10
40/40 [==============================] - 9s 230ms/step - loss: 0.1418 - accuracy: 0.9675 - val_loss: 0.0953 - val_accuracy: 0.9795
Epoch 4/10
40/40 [==============================] - 9s 231ms/step - loss: 0.1807 - accuracy: 0.9609 - val_loss: 0.0792 - val_accuracy: 0.9758
Epoch 5/10
40/40 [==============================] - 9s 232ms/step - loss: 0.1106 - accuracy: 0.9670 - val_loss: 0.0832 - val_accuracy: 0.9777
Epoch 6/10
40/40 [==============================] - 9s 230ms/step - loss: 0.1170 - accuracy: 0.9717 - val_loss: 0.1145 - val_accuracy: 0.9665
Epoch 7/10
40/40 [==============================] - 9s 232ms/step - loss: 0.1199 - accuracy: 0.9758 - val_loss: 0.0523 - val_accuracy: 0.9888
Epoch

In [44]:
# save (위에서 load)
# save_format='h5' 로 저장
from keras.models import load_model
resume_model.save('E:/##kpu_capstone_voice_data/##model/saddummy@gmail.com_augment_mk_TV_spinoff.h5')

# test_h5
# resume_model

In [154]:
file_directory = "E:/##kpu_capstone_voice_data/!snowflo_real_final/validation_final_not_augment"

from keras.preprocessing import image

for (dirpath, dirnames, filenames) in os.walk(file_directory):
    for filename in filenames:
        print("=="+ filename + "==\n")
        image_path = file_directory +'/'+ filename
        print(image_path)
        img = image.load_img(image_path, target_size=(img_height, img_width))
        img_array = keras.preprocessing.image.img_to_array(img)
        img_array = tf.expand_dims(img_array, 0) # Create a batch
        
        predictions = resume_model.predict(img_array)
        print(predictions)
        score = tf.nn.softmax(predictions[0])
        print(score)
        print("새로운 데이터는 {} 클래스일 확률이 {:.2f}%입니다..".format(class_names[np.argmax(score)], 100 * np.max(score)))
        print("=============================================================================")


==1_call_sangho (1).png==

E:/##kpu_capstone_voice_data/!snowflo_real_final/validation_final_not_augment/1_call_sangho (1).png
[[2.04410644e-09 4.77801716e-22 9.99999523e-01 6.04490893e-12
  4.96793939e-09 1.06984796e-10 3.03593205e-16 4.23914855e-07]]
tf.Tensor(
[0.10289887 0.10289887 0.27970794 0.10289887 0.10289887 0.10289887
 0.10289887 0.1028989 ], shape=(8,), dtype=float32)
새로운 데이터는 3상호한테전화해줘 클래스일 확률이 27.97%입니다..
==1_call_sangho (2).png==

E:/##kpu_capstone_voice_data/!snowflo_real_final/validation_final_not_augment/1_call_sangho (2).png
[[4.8755213e-05 4.0535275e-10 8.6474407e-01 3.6122105e-03 1.5843513e-03
  1.5243178e-04 5.3425692e-10 1.2985815e-01]]
tf.Tensor(
[0.10506402 0.10505889 0.24945165 0.10543907 0.10522547 0.1050749
 0.10505889 0.11962707], shape=(8,), dtype=float32)
새로운 데이터는 3상호한테전화해줘 클래스일 확률이 24.95%입니다..
==1_call_sangho (3).png==

E:/##kpu_capstone_voice_data/!snowflo_real_final/validation_final_not_augment/1_call_sangho (3).png
[[3.4183259e-07 6.0546270e-08 1.6333

tf.Tensor(
[0.10289907 0.10289907 0.2797053  0.10289907 0.1029003  0.10289907
 0.10289907 0.10289907], shape=(8,), dtype=float32)
새로운 데이터는 3상호한테전화해줘 클래스일 확률이 27.97%입니다..
==1_call_sangho (20).png==

E:/##kpu_capstone_voice_data/!snowflo_real_final/validation_final_not_augment/1_call_sangho (20).png
[[9.2274563e-11 2.2840777e-22 9.9999237e-01 3.7306725e-13 7.6026186e-06
  1.5272753e-14 2.3904755e-19 1.8197955e-08]]
tf.Tensor(
[0.10289899 0.10289899 0.2797063  0.10289899 0.10289977 0.10289899
 0.10289899 0.10289899], shape=(8,), dtype=float32)
새로운 데이터는 3상호한테전화해줘 클래스일 확률이 27.97%입니다..
==1_call_sangho (21).png==

E:/##kpu_capstone_voice_data/!snowflo_real_final/validation_final_not_augment/1_call_sangho (21).png
[[3.5995750e-08 1.5746864e-20 9.1969377e-01 5.1106164e-11 8.0289625e-02
  1.0216244e-11 3.5376599e-14 1.6533982e-05]]
tf.Tensor(
[0.10425204 0.10425203 0.2615185  0.10425203 0.11296758 0.10425203
 0.10425203 0.10425375], shape=(8,), dtype=float32)
새로운 데이터는 3상호한테전화해줘 클래스일 확률이 26.15%입니

[[9.9999750e-01 4.2067281e-22 2.7582807e-08 1.2885966e-13 2.4884012e-06
  4.3625537e-16 4.4766526e-12 3.2752144e-08]]
tf.Tensor(
[0.2797075  0.1028989  0.1028989  0.1028989  0.10289916 0.1028989
 0.1028989  0.10289891], shape=(8,), dtype=float32)
새로운 데이터는 1굿모닝 클래스일 확률이 27.97%입니다..
==2_good_morning (21).png==

E:/##kpu_capstone_voice_data/!snowflo_real_final/validation_final_not_augment/2_good_morning (21).png
[[1.0000000e+00 1.0510316e-19 7.1053458e-12 1.8684954e-15 1.2970389e-08
  9.0305902e-17 6.7893824e-14 9.9528732e-11]]
tf.Tensor(
[0.27970806 0.10289886 0.10289886 0.10289886 0.10289886 0.10289886
 0.10289886 0.10289886], shape=(8,), dtype=float32)
새로운 데이터는 1굿모닝 클래스일 확률이 27.97%입니다..
==2_good_morning (22).png==

E:/##kpu_capstone_voice_data/!snowflo_real_final/validation_final_not_augment/2_good_morning (22).png
[[9.9999845e-01 3.0464930e-19 4.5609285e-09 3.6791750e-12 1.3955537e-06
  3.5300769e-13 1.2059581e-13 1.0495199e-07]]
tf.Tensor(
[0.2797077  0.10289888 0.10289888 0.10289888

[[1.4976648e-03 9.7069840e-14 1.2911046e-05 2.1503096e-05 1.6710109e-07
  9.9742162e-01 5.3446370e-10 1.0462097e-03]]
tf.Tensor(
[0.10309996 0.10294567 0.10294699 0.10294788 0.10294569 0.27911472
 0.10294567 0.10305343], shape=(8,), dtype=float32)
새로운 데이터는 6음악추천해줘 클래스일 확률이 27.91%입니다..
==3_recommend_music (12).png==

E:/##kpu_capstone_voice_data/!snowflo_real_final/validation_final_not_augment/3_recommend_music (12).png
[[5.3829964e-02 1.2202084e-08 4.2065047e-05 5.1079287e-06 2.1147739e-09
  9.4580412e-01 1.4308280e-07 3.1855336e-04]]
tf.Tensor(
[0.10957887 0.10383618 0.10384056 0.10383672 0.10383618 0.26736602
 0.10383619 0.10386927], shape=(8,), dtype=float32)
새로운 데이터는 6음악추천해줘 클래스일 확률이 26.74%입니다..
==3_recommend_music (13).png==

E:/##kpu_capstone_voice_data/!snowflo_real_final/validation_final_not_augment/3_recommend_music (13).png
[[1.9025259e-12 3.2987086e-15 9.5172432e-11 8.4709170e-11 4.8961706e-11
  1.0000000e+00 6.5163980e-21 3.3040980e-08]]
tf.Tensor(
[0.10289886 0.10289886 0.

  2.3023907e-07 1.6590354e-08 2.4302862e-09]]
tf.Tensor(
[0.10290346 0.2796496  0.10290346 0.10292958 0.10290346 0.10290349
 0.10290346 0.10290346], shape=(8,), dtype=float32)
새로운 데이터는 2내일날씨어때 클래스일 확률이 27.96%입니다..
==4_today_weather (6).png==

E:/##kpu_capstone_voice_data/!snowflo_real_final/validation_final_not_augment/4_today_weather (6).png
[[2.3307378e-10 8.1207711e-01 9.7833660e-12 1.8791872e-01 4.8478581e-16
  1.0132585e-10 2.9320224e-06 1.1871051e-06]]
tf.Tensor(
[0.10571583 0.23813356 0.10571583 0.12757103 0.10571583 0.10571583
 0.10571613 0.10571595], shape=(8,), dtype=float32)
새로운 데이터는 2내일날씨어때 클래스일 확률이 23.81%입니다..
==4_today_weather (7).png==

E:/##kpu_capstone_voice_data/!snowflo_real_final/validation_final_not_augment/4_today_weather (7).png
[[9.68152850e-11 9.99837756e-01 2.35700708e-16 1.62192504e-04
  7.37254067e-23 2.03043884e-11 3.07846992e-09 1.02993315e-10]]
tf.Tensor(
[0.1029018  0.27967072 0.1029018  0.1029185  0.1029018  0.1029018
 0.1029018  0.1029018 ], shape=(8,)

[[8.8686433e-15 4.5609876e-09 1.8034023e-13 1.0000000e+00 2.8226645e-23
  1.6273521e-14 7.6133110e-14 3.8552153e-10]]
tf.Tensor(
[0.10289886 0.10289886 0.10289886 0.27970806 0.10289886 0.10289886
 0.10289886 0.10289886], shape=(8,), dtype=float32)
새로운 데이터는 4오늘날씨어때 클래스일 확률이 27.97%입니다..
==4_today_weather (24).png==

E:/##kpu_capstone_voice_data/!snowflo_real_final/validation_final_not_augment/4_today_weather (24).png
[[6.2292532e-18 3.2669853e-09 4.5532229e-14 1.0000000e+00 8.1632549e-24
  2.8749986e-17 8.3023849e-15 1.1794916e-10]]
tf.Tensor(
[0.10289886 0.10289886 0.10289886 0.27970806 0.10289886 0.10289886
 0.10289886 0.10289886], shape=(8,), dtype=float32)
새로운 데이터는 4오늘날씨어때 클래스일 확률이 27.97%입니다..
==5_tomorrow_weather (20).png==

E:/##kpu_capstone_voice_data/!snowflo_real_final/validation_final_not_augment/5_tomorrow_weather (20).png
[[5.7669225e-09 9.9029070e-01 4.2264434e-13 9.7057186e-03 2.5611214e-17
  4.9201725e-14 3.5511896e-06 1.7884945e-08]]
tf.Tensor(
[0.10307395 0.2774768  0.10

[[2.2471868e-07 9.9999976e-01 1.6741226e-11 3.4398630e-11 6.9139411e-23
  1.8541410e-12 3.1265024e-08 1.1993918e-10]]
tf.Tensor(
[0.10289888 0.279708   0.10289886 0.10289886 0.10289886 0.10289886
 0.10289886 0.10289886], shape=(8,), dtype=float32)
새로운 데이터는 2내일날씨어때 클래스일 확률이 27.97%입니다..
==5_tomorrow_weather (37).png==

E:/##kpu_capstone_voice_data/!snowflo_real_final/validation_final_not_augment/5_tomorrow_weather (37).png
[[5.1856275e-12 1.0000000e+00 1.4508829e-17 5.9283030e-14 2.7033356e-25
  5.7183618e-16 4.7985124e-15 2.9691844e-15]]
tf.Tensor(
[0.10289886 0.27970806 0.10289886 0.10289886 0.10289886 0.10289886
 0.10289886 0.10289886], shape=(8,), dtype=float32)
새로운 데이터는 2내일날씨어때 클래스일 확률이 27.97%입니다..
==5_tomorrow_weather (38).png==

E:/##kpu_capstone_voice_data/!snowflo_real_final/validation_final_not_augment/5_tomorrow_weather (38).png
[[1.9601467e-08 9.6523005e-01 1.3058542e-11 3.4575678e-02 2.3608377e-15
  3.0450536e-14 1.9396491e-04 2.8101715e-07]]
tf.Tensor(
[0.10351145 0.2717580

[[3.5216923e-12 1.5401338e-16 1.2176174e-15 2.4544893e-12 4.7739245e-06
  2.8908122e-18 9.9999273e-01 2.5415611e-06]]
tf.Tensor(
[0.10289898 0.10289898 0.10289898 0.10289898 0.10289946 0.10289898
 0.2797064  0.10289925], shape=(8,), dtype=float32)
새로운 데이터는 7출근길교통상황어때 클래스일 확률이 27.97%입니다..
==6_working_path (14).png==

E:/##kpu_capstone_voice_data/!snowflo_real_final/validation_final_not_augment/6_working_path (14).png
[[4.9399512e-12 7.7248930e-15 2.0578380e-15 8.8793113e-12 1.7022553e-07
  1.9639160e-16 9.9999523e-01 4.7079716e-06]]
tf.Tensor(
[0.10289894 0.10289894 0.10289894 0.10289894 0.10289896 0.10289894
 0.27970698 0.10289942], shape=(8,), dtype=float32)
새로운 데이터는 7출근길교통상황어때 클래스일 확률이 27.97%입니다..
==6_working_path (15).png==

E:/##kpu_capstone_voice_data/!snowflo_real_final/validation_final_not_augment/6_working_path (15).png
[[1.0421654e-07 7.6111872e-05 2.6227745e-10 3.9407937e-06 5.9352576e-05
  1.7725524e-04 9.2705876e-01 7.2624490e-02]]
tf.Tensor(
[0.10413729 0.10414521 0.104137

tf.Tensor(
[0.10353393 0.10353393 0.10353393 0.10353394 0.2714573  0.10353515
 0.10353393 0.10733794], shape=(8,), dtype=float32)
새로운 데이터는 5유튜브에서동빈나틀어줘 클래스일 확률이 27.15%입니다..
==7_youtube_dongbinna (14).png==

E:/##kpu_capstone_voice_data/!snowflo_real_final/validation_final_not_augment/7_youtube_dongbinna (14).png
[[3.1086624e-19 2.6570893e-30 3.4401094e-08 9.7468894e-15 9.9999738e-01
  3.4552641e-17 4.8550373e-14 2.5749287e-06]]
tf.Tensor(
[0.1028989  0.1028989  0.10289891 0.1028989  0.2797075  0.1028989
 0.1028989  0.10289918], shape=(8,), dtype=float32)
새로운 데이터는 5유튜브에서동빈나틀어줘 클래스일 확률이 27.97%입니다..
==7_youtube_dongbinna (15).png==

E:/##kpu_capstone_voice_data/!snowflo_real_final/validation_final_not_augment/7_youtube_dongbinna (15).png
[[8.1559685e-21 9.3550146e-26 6.3590348e-11 7.6165906e-15 9.9996388e-01
  5.4142578e-07 1.2419989e-16 3.5503035e-05]]
tf.Tensor(
[0.10289951 0.10289951 0.10289951 0.10289951 0.27969974 0.10289955
 0.10289951 0.10290316], shape=(8,), dtype=float32)
새로운 데이터

In [ ]:
# 그래프
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss=history.history['loss']
val_loss=history.history['val_loss']

epochs_range = range(EPOCHS)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

### ========================= 이하 부분은 back up 본에서 찾을것 =========================

### 참조
https://github.com/keras-team/keras/issues/7924

weight 계속 유지 하려면 tf로 저장

new class 추가 하려면 가장 아래 dense layer 삭제 -> 새로운 dense layer 추가